<div style="text-align: center; padding: 30px">
  <h1><strong>Ejercicios de Modelado</strong></h1>
  <h3><strong>Ejercicio 2</strong></h3>
</div>

Se deben procesar varios trabajos en una máquina y no se permiten prioridades para procesar los trabajos. Sea $N = \{1, 2, \dots , n\}$ el conjunto de trabajos a procesar. Sean también $p_{j}$ y $d_{j}$ , el tiempo de procesamiento y la fecha de vencimiento del trabajo $j ∈ N$ (expresados en días), respectivamente. Se requiere definir el orden en el que se procesarán los trabajos para minimizar la máxima tardanza (retraso). Definimos la tardanza de un trabajo como la diferencia entre su fecha de finalización y su fecha de entrega. Sea $C_{j}$ el tiempo de finalización del trabajo $j ∈ N$ , y sea $y_{jk}$ una variable de decisión binaria, que es igual a 1 si el trabajo $j$ se procesa antes que el trabajo $k$ e igual a 0 en caso contrario. Además, sea $L_{máx}$ la tardanza máxima.

### **Uso de Python y el Solver SCIP**



En Python, utilizamos el solver `SCIP` para definir y resolver este problema. Para declarar variables binarias, se utiliza el método:

```python
solver.BoolVar(name)
```

In [1]:
from ortools.linear_solver import pywraplp

In [2]:
solver = pywraplp.Solver.CreateSolver('SCIP')

### **Conjuntos de Índices**

- $N$: Conjunto de trabajos, $C = \{1, 2, \dots, 10\}$

### **Parámetros**

Los parámetros capturan la información del problema.

- $ \forall j \in N: p_{j} = \text {tiempo de procesamiento} $
- $ \forall j \in N: d_{j} = \text {fecha de vencimiento} $

Donde se define la tardanza de un trabajo como la diferencia entre su fecha de finalización y su fecha de entrega

In [3]:
# Conjuntos
N = range(1, 11)  # Trabajos {1, 2, ..., 10}

# Fecha de vencimiento
d = [21, 25, 23, 24, 27, 30, 28, 31, 29, 22] 
#El último número de 'd' no estaba en la presentación, se incluyó para poder resolver el problema
#Tiempo de procesamiento
p = [3, 4, 2, 5, 6, 1, 4, 5, 7, 10]

### **Variables de Decisión**

Las variables de decisión están relacionadas con la respuesta al problema.

$$ \forall j \in N: C_{j} = \text {tiempo de finalización} $$

In [4]:
# Variable de tiempo de finalización para cada trabajo
C = {}
for j in N:
    C[j] = solver.NumVar(0, solver.infinity(), f'C[{j}]')

$$
y_{j,k} = 
\begin{cases} 
1   \text{ si el trabajo $j$ se procesa antes que el trabajo $k; \quad \forall j \in N, \forall k \in N$} \\ 
0   \text{ en otro caso}.
\end{cases}
$$


In [5]:
x = {}  # x[j, k] = 1 si el curso i se programa el día d en el periodo p
for j in N: # Trabajos
    for k in N: # Trabajos
        x[j, k] = solver.BoolVar(f'x[{j},{k}]') 
        # Variable binaria (0, 1)


$$
L_{máx} = \text{ la tardanza máxima }
$$


In [6]:
L = solver.IntVar(0, solver.infinity(), 'L') #en días
# Int.Var es una variable entera desde 0 a infinito con nombre L

### **Función Objetivo**

El objetivo es minimizar la tardanza máxima

$$
\min L
$$

In [7]:
solver.Minimize(L)

### **Restricciones**

El tiempo de finalización de cada trabajo es mayor o igual a su tiempo
de procesamiento
$$
C_{j} \geq p_{j} \quad \forall j \in N
$$

In [8]:
for j in N:
    solver.Add(C[j] >= p[j-1]) 

Para cada par $(j, k)$ de trabajos, el trabajo $j$ se procesa antes que el
trabajo $k$ o el trabajo $k$ se procesa antes que el trabajo $j$.
$$
C_{j} + p_{k} \leq C_{k} + M(1-y_{jk})
$$

In [9]:
M = sum(p) # M es la suma de los tiempos de procesamiento de todos los trabajos, una cota superior para C[j]

for j in N:
    for k in N:
        if j != k:
            solver.Add(C[j] + p[k-1] <= C[k] + M*(1 - x[j, k]))

$$
C_{k} + p_{j} \leq C_{j} + M y_{jk}
$$

In [10]:
for j in N:
    for k in N:
        if j != k:
            solver.Add(C[k] + p[j-1] <= C[j] + M*(x[j, k]))

$L_{máx}$ debe ser igual a la tardanza máxima.

$$
L_{máx} \geq C_{j} - d_{j} \quad \forall j \in N
$$

In [11]:
for j in N: # Trabajos 
     solver.Add(L >= C[j] - d[j-1])

### **Resolver**

In [12]:
solver.Solve()

0

In [13]:
print(f'La tardanza más grande: {solver.Objective().Value()}')

La tardanza más grande: 16.000000000000085
